### Expressivity

In [1]:
import numpy as np
from ansatz import Ansatz
from utils import pennylane_to_qiskit
import pennylane as qml

n_qubits = 4
results = {}
depth = 1
circuits = {}
for ansatz_id in range(1, 20):
    results[ansatz_id] = {}
    super_ansatz = Ansatz(ansatz_id, n_qubits, depth)
    ansatz = super_ansatz.get_circuit()
    param_shape = super_ansatz.get_params_shape()
    if param_shape[1] is not None:
        params1 = np.random.uniform(0, 2 * np.pi, param_shape[0])
        params2 = np.random.uniform(0, 2 * np.pi, param_shape[1])
        params = [params1, params2]

        @qml.qnode(qml.device("default.qubit", wires=n_qubits))
        def circuit(params1, params2):
            ansatz(params1, params2)
            return qml.expval(qml.PauliZ(0))
    else:
        params1 = np.random.uniform(0, 2 * np.pi, param_shape[0])
        params = [params1]
        @qml.qnode(qml.device("default.qubit", wires=n_qubits))
        def circuit(params1):
            ansatz(params1)
            return qml.expval(qml.PauliZ(0))

    qiskit_circuit = pennylane_to_qiskit(circuit, n_qubits, params=params)
    circuits[ansatz_id] = qiskit_circuit.remove_final_measurements(inplace=False)

    # Draw the circuit in file
    #filename = f"figures\\ansatz_{ansatz_id}_depth_{depth}.png"
    #qiskit_circuit.draw(output="mpl", filename=filename)

In [ ]:
from qleet.analyzers.expressibility import Expressibility
from qleet.interface.circuit import CircuitDescriptor

n_shots = max(2**n_qubits, 1024)
print(f"Number of shots: {n_shots}")

for ansatz_id in range(1, 20):
    qiskit_circuit = circuits[ansatz_id]
    params = qiskit_circuit.parameters
    print(f"Ansatz {ansatz_id} parameters: {params}")
    circuit_descriptor = CircuitDescriptor(qiskit_circuit, params)
    exp = Expressibility(circuit_descriptor, samples=n_shots)
    results[ansatz_id]["expressivity"] = exp.expressibility()
    print(f"Ansatz {ansatz_id}: {results[ansatz_id]['expressivity']}")

Number of shots: 1024
Ansatz 1 parameters: ParameterView([Parameter(x0), Parameter(x1), Parameter(x2), Parameter(x3), Parameter(x4), Parameter(x5), Parameter(x6), Parameter(x7)])
Ansatz 1: 0.4044084774256419
Ansatz 2 parameters: ParameterView([Parameter(x0), Parameter(x1), Parameter(x2), Parameter(x3), Parameter(x4), Parameter(x5), Parameter(x6), Parameter(x7)])
Ansatz 2: 0.40913938946783923
Ansatz 3 parameters: ParameterView([Parameter(x0), Parameter(x1), Parameter(x10), Parameter(x2), Parameter(x3), Parameter(x4), Parameter(x5), Parameter(x6), Parameter(x7), Parameter(x8), Parameter(x9)])
Ansatz 3: 0.3251019146295814
Ansatz 4 parameters: ParameterView([Parameter(x0), Parameter(x1), Parameter(x10), Parameter(x2), Parameter(x3), Parameter(x4), Parameter(x5), Parameter(x6), Parameter(x7), Parameter(x8), Parameter(x9)])
Ansatz 4: 0.1591163670095324
Ansatz 5 parameters: ParameterView([Parameter(x0), Parameter(x1), Parameter(x10), Parameter(x11), Parameter(x12), Parameter(x13), Parameter(x